In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [457 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [979 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,236 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_revised_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_revised_df.show()




+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_revised_df.createOrReplaceTempView('home_sales')


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
  SELECT ROUND(AVG(price),2) AS Average_Price, YEAR(date) as year FROM home_sales
    where bedrooms=4
    group by year
    order by year
  """).show()


+-------------+----+
|Average_Price|year|
+-------------+----+
|     300263.7|2019|
|    298353.78|2020|
|    301819.44|2021|
|    296363.88|2022|
+-------------+----+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
  SELECT ROUND(AVG(price),2) AS Average_Price, date_built as year FROM home_sales
    where bedrooms=3 AND bathrooms=3
    group by year
    order by year
  """).show()


+-------------+----+
|Average_Price|year|
+-------------+----+
|    292859.62|2010|
|    291117.47|2011|
|    293683.19|2012|
|    295962.27|2013|
|    290852.27|2014|
|     288770.3|2015|
|    290555.07|2016|
|    292676.79|2017|
+-------------+----+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
  SELECT ROUND(AVG(price),2) AS Average_Price, date_built as year FROM home_sales
    where bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
    group by year
    order by year
  """).show()


+-------------+----+
|Average_Price|year|
+-------------+----+
|    285010.22|2010|
|    276553.81|2011|
|    307539.97|2012|
|    303676.79|2013|
|    298264.72|2014|
|    297609.97|2015|
|     293965.1|2016|
|    280317.58|2017|
+-------------+----+



In [21]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
  SELECT ROUND(AVG(price),2) AS Average_Price, view FROM home_sales
    where price >= 350000
    group by view
    order by Average_Price
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|view|
+-------------+----+
|    393973.32|  33|
|    394964.36|  37|
|    395655.38|   6|
|     396964.5|  42|
|    397389.25|   2|
|    397771.65|  29|
|     397862.0|  30|
|    398418.37|  38|
|     398447.5|  47|
|    398474.49|  17|
|    398570.03|  14|
|    398592.71|   8|
|     398867.6|   3|
|    398917.98|  13|
|    398953.17|  19|
|    399234.22|  39|
|    399332.91|  18|
|    399367.72|  32|
|    399522.81|  20|
|    399537.66|  27|
+-------------+----+
only showing top 20 rows

--- 0.521233320236206 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""
  SELECT ROUND(AVG(price),2) AS Average_Price, view FROM home_sales
    where price >= 350000
    group by view
    order by Average_Price
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|Average_Price|view|
+-------------+----+
|    393973.32|  33|
|    394964.36|  37|
|    395655.38|   6|
|     396964.5|  42|
|    397389.25|   2|
|    397771.65|  29|
|     397862.0|  30|
|    398418.37|  38|
|     398447.5|  47|
|    398474.49|  17|
|    398570.03|  14|
|    398592.71|   8|
|     398867.6|   3|
|    398917.98|  13|
|    398953.17|  19|
|    399234.22|  39|
|    399332.91|  18|
|    399367.72|  32|
|    399522.81|  20|
|    399537.66|  27|
+-------------+----+
only showing top 20 rows

--- 0.5051395893096924 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_revised_df.write.partitionBy("date_built").mode("overwrite").parquet("p_date_built")

In [23]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('p_date_built')

In [24]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_date_built')

In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""
  SELECT ROUND(AVG(price),2) AS Average_Price, view FROM p_date_built
    where price >= 350000
    group by view
    order by Average_Price
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|view|
+-------------+----+
|    393973.32|  33|
|    394964.36|  37|
|    395655.38|   6|
|     396964.5|  42|
|    397389.25|   2|
|    397771.65|  29|
|     397862.0|  30|
|    398418.37|  38|
|     398447.5|  47|
|    398474.49|  17|
|    398570.03|  14|
|    398592.71|   8|
|     398867.6|   3|
|    398917.98|  13|
|    398953.17|  19|
|    399234.22|  39|
|    399332.91|  18|
|    399367.72|  32|
|    399522.81|  20|
|    399537.66|  27|
+-------------+----+
only showing top 20 rows

--- 0.8413138389587402 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [28]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False